In [1]:
!pip install -q beautifulsoup4==4.12.2 PyMuPDFb==1.24.10 PyMuPDF==1.24.13 tqdm==4.66.5 duckdb==0.10.0 tabulate==0.9.0 plotly==5.20.0 python-dotenv openai==1.42.0

In [2]:
SYMBOL = "PTT"

In [3]:
import requests
import pandas as pd
import re


session = requests.Session()
session.get(f"https://www.settrade.com/th/equities/quote/{SYMBOL}/financial-statement/full")


headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-TH,en;q=0.9,th-TH;q=0.8,th;q=0.7,en-GB;q=0.6,en-US;q=0.5',
    # 'cookie': 'visid_incap_2685215=l/JAPqmmR8KSk7rLhs3BzKvPSGYAAAAAQUIPAAAAAABALhVFFEmAe3vRYqQgqqQo; exp_history={"go_expid":"5AD93i4KR9-ZVNOhL9Vr2w-V2","msgt":"popup","count":1}; _cc_id=4d4cfaa1a003693b7c5cc45baf0913e9; __gads=ID=f9d48026689a99be:T=1716047793:RT=1716047793:S=ALNI_Ma_zJhrDoYFiXyfutD1XSHxPWA3Uw; __gpi=UID=00000e2256455e78:T=1716047793:RT=1716047793:S=ALNI_Mby0wdaHRd_h-eJjYJY5wLHS0TJAQ; __eoi=ID=2567fce2f1c28622:T=1716047793:RT=1716047793:S=AA-AfjaTaPE5D-KD9jKsPW8xY0gE; cto_bundle=44weE19pQlBrWWZRQUdySnclMkJmNXlkNkk4blI2MTgwTGNzSUZGSSUyRjRjdGRsUUZMZ1pWSDViOHIlMkY1c3lHc0NuZjBReDUlMkZVd21FT3piZkJ0MmE3bEEyRCUyQlVqYlFWbWpGVkQ0S1gxMlZUQTdIZXV3T05iN2VqbGZKWlNqRnloTm5jT3ltTnBCS1kzc3lxVWFUVEcxWFhHd2FnUjdCb1VMRWFzcmJKaG1SWm0lMkZqJTJGcE03c0UwJTJGYXRHU3FpUiUyQlhSRWlhMkxNd2o; cto_bidid=b-mYHF9lQlh6WHpHZHNta0hieDFuZ21FQ2dKQjdWRnR2RHNydllBQ2hkZEhOb1RxRzFhM1hGU29zbDgxcVdHOTFCQm1jJTJCb0ZaOVdGdjBPcCUyQnVmTGVVTUJHUmtCNDVkNTBOM1ZMdG9sVWg2OER1TFZDaENRNFElMkZjTjYwaFpGY0lsV3hjUUozZ2NUJTJGRG1KWWl5QlNqWWE5TVlKUSUzRCUzRA; visid_incap_2868064=9TnELvraSEyQ3rCeC/3tp6/imGYAAAAAQUIPAAAAAABQXj/9u+lgqIuVKksYLcer; _ga_W7C0TNQT63=GS1.1.1721377038.3.1.1721377082.0.0.0; charlot=6a8621f0-cdcf-45a6-b1c2-f81785e2d02f; _gcl_au=1.1.1685208698.1724255273; _gid=GA1.2.1052211754.1724255275; api_call_counter=5; SET_COOKIE_POLICY=20231111093657; nlbi_2685215=IC0uOhgeyBy9J7BzwZdY4QAAAAAqEQh9oBWiKDUUC927BZYM; _cbclose=1; _cbclose64035=1; route=0f35441e6abd8a06b9ca7c7f54fa2b5e; incap_ses_235_2685215=SjIpB/f0VXZxsG4/XeNCAyHexmYAAAAA4/zIE2DbjM+Hy9alp/2MXA==; landing_url=https://www.settrade.com/th/equities/quote/OR/overview?_gl=1%2a9iyx2w%2a_gcl_au%2aMTYwODA1NzI0NS4xNzIzNjQzNjg2%2a_ga%2aMTgyMzgyMTUxNS4xNzE1NTk1MjY5%2a_ga_ET2H60H2CB%2aMTcyNDI1NTIxMS4yNi4xLjE3MjQyNTU0ODguNDkuMC4w; _uid64035=DB23855A.7; _ctout64035=1; _gat_UA-29750633-11=1; _ga=GA1.1.1767658003.1716047789; _ga_W11E901KXL=GS1.1.1724324251.9.1.1724324280.31.0.0; _ga_6WS2P0P25V=GS1.1.1724324251.9.1.1724324280.31.0.0',
    'if-none-match': '"342c523b"',
    'priority': 'u=1, i',
    'referer': 'https://www.settrade.com/th/equities/quote/OR/financial-statement/full',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

def get_sheet(period: str) -> pd.DataFrame:
    params = {
        'accountType': 'balance_sheet',
        'fsType': 'company',
        'period': period,
#         'language': 'th',
        'language': 'ENG',
    }

    response = session.get(
        f'https://www.settrade.com/api/set/stock/{SYMBOL}/financialstatement',
        params=params,
        headers=headers,
    )
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data["accounts"])

def get_business_type(symbol: str) -> str:
    response = session.get(f"https://www.settrade.com/th/equities/quote/{symbol}/overview")

    response.raise_for_status()
    text = response.text
    pattern = r'businessType:\s*"([^"]+)"'
    found = re.search(pattern, text)

    if found:
        target_data = found.group(1)
        return target_data
    else:
        print("No match found.")
        return None
    return response.json()

In [4]:
from_ = 2019
to = 2024

year_range = range(from_, to)
first_year, *year_range = year_range

In [5]:
sheet = get_sheet(f"YE_{first_year}")[["accountCode", "accountName", "amount"]]
sheet[str(first_year)] = sheet["amount"]
sheet = sheet.drop("amount", axis=1)
for year in year_range:
    sheet_by_year = get_sheet(f"YE_{year}")[["accountCode", "accountName", "amount"]]
    sheet_by_year[str(year)] = sheet_by_year["amount"]
    sheet_by_year = sheet_by_year.drop("amount", axis=1)

    sheet = pd.merge(sheet, sheet_by_year, on="accountCode", how='outer', suffixes=("", f"_{year}"))
    sheet['accountName'] = sheet['accountName'].fillna(sheet[f'accountName_{year}'])
    sheet = sheet.drop(f'accountName_{year}', axis=1)

sheet


,accountCode,accountName,2019,2020,2021,2022,2023
0,100000,สินทรัพย์,NaN,NaN,NaN,NaN,NaN
1,109999,รวมสินทรัพย์,8.896762e+08,8.993883e+08,1.004906e+09,1.081772e+09,1.117391e+09
2,110000,สินทรัพย์หมุนเวียน,NaN,NaN,NaN,NaN,NaN
3,110100,เงินสดและรายการเทียบเท่าเงินสด,4.648081e+07,5.634189e+07,4.585339e+07,5.278798e+07,8.757027e+07
4,111600,เงินลงทุนระยะสั้น - สุทธิ,1.500234e+07,4.440431e+07,5.533088e+06,3.468734e+06,7.253800e+06
...,...,...,...,...,...,...,...
81,319921,ส่วนเกิน (ต่ำกว่า) ทุนจากการเปลี่ยนแปลงส่วนได้...,NaN,1.558256e+06,NaN,NaN,NaN
82,319990,องค์ประกอบอื่นของส่วนของผู้ถือหุ้น - อื่นๆ,NaN,NaN,5.247456e+06,3.240285e+06,2.055925e+06
83,319999,รวมส่วนของผู้ถือหุ้นของบริษัทใหญ่,6.073067e+08,6.230825e+08,6.375067e+08,6.287935e+08,6.627900e+08
84,900020,เงินให้กู้ยืมระยะยาว - ส่วนที่ถึงกำหนดชำระเกิน...,2.498189e+07,4.466327e+07,NaN,NaN,NaN


In [6]:
sheet["accountName"].unique()

array(['สินทรัพย์', 'รวมสินทรัพย์', 'สินทรัพย์หมุนเวียน',
       'เงินสดและรายการเทียบเท่าเงินสด', 'เงินลงทุนระยะสั้น - สุทธิ',
       'ลูกหนี้การค้าและลูกหนี้หมุนเวียนอื่น - สุทธิ',
       'เงินให้กู้ยืมระยะสั้นและดอกเบี้ยค้างรับ',
       'บุคคลหรือกิจการที่เกี่ยวข้องกัน', 'สินค้าคงเหลือ - สุทธิ',
       'สินค้าสำเร็จรูป', 'วัตถุดิบ อะไหล่และวัสดุสิ้นเปลือง',
       'สินทรัพย์อนุพันธ์ - หมุนเวียน',
       'สินทรัพย์ทางการเงินหมุนเวียนอื่น', 'ลูกหนี้หมุนเวียนอื่น',
       'สินทรัพย์ทางการเงินหมุนเวียนอื่น - อื่น ๆ',
       'สินทรัพย์หมุนเวียนอื่น', 'สินทรัพย์หมุนเวียนอื่น - อื่น ๆ',
       'รวมสินทรัพย์หมุนเวียน', 'สินทรัพย์ไม่หมุนเวียน',
       'เงินลงทุนระยะยาว - สุทธิ',
       'เงินให้กู้ยืมระยะยาวและดอกเบี้ยค้างรับ - ส่วนที่ถึงกำหนดชำระเกินกว่าหนึ่งปี',
       'เงินลงทุนในบริษัทย่อย บริษัทร่วม และการร่วมค้า ที่บันทึกด้วยวิธีอื่น - สุทธิ',
       'เงินลงทุนในบริษัทย่อย', 'เงินลงทุนในการร่วมค้า',
       'สินทรัพย์อนุพันธ์ - ไม่หมุนเวียน',
       'สินทรัพย์ทางการเงินไม่หมุนเวียนอื่น',

In [7]:
business_type = get_business_type(SYMBOL)

In [15]:
from string import Template


template = Template("""
# Context:

Stock name: ${SYMBOL}
Business type: ${business_type}

Balance sheet: ${sheet}
Structure:

${task}
""")
sheet_prompt = sheet.drop("accountCode", axis=1).astype(str).to_markdown(index=False, floatfmt=",.1f")
print(
    template.safe_substitute(
        SYMBOL=SYMBOL,
        business_type=business_type,
        sheet=sheet_prompt,
    )
)


# Context:

Stock name: PTT
Business type: ประกอบด้วยกิจการที่ ปตท. ดำเนินการเอง ได้แก่ ธุรกิจก๊าซธรรมชาติ ธุรกิจระบบท่อส่งก๊าซธรรมชาติ ธุรกิจการค้าระหว่างประเทศธุรกิจใหม่และโครงสร้างพื้นฐาน และธุรกิจที่ลงทุนผ่านบริษัทย่อยและ\u002Fหรือกิจการที่ควบคุมร่วมกันและบริษัทร่วม (กลุ่มบริษัท) ได้แก่ ธุรกิจสำรวจและผลิตปิโตรเลียม ธุรกิจก๊าซธรรมชาติเหลว ธุรกิจปิโตรเคมีและการกลั่น ธุรกิจน้ำมันและค้าปลีก ธุรกิจไฟฟ้าและสาธารณูปการ ธุรกิจให้บริการ

Balance sheet: | accountName                                                      |          2019 |          2020 |            2021 |            2022 |            2023 |
|:-----------------------------------------------------------------|--------------:|--------------:|----------------:|----------------:|----------------:|
| สินทรัพย์                                                           |         nan   |         nan   |           nan   |           nan   |           nan   |
| รวมสินทรัพย์                                                        | 889,676

# Set up LLM

In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
from typing_extensions import TypedDict
from IPython.display import Markdown
from google.colab import userdata


GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)


MODEL = "typhoon-v1.5x-70b-instruct"



TYPHOON_API_KEY = userdata.get('TYPHOON_API_KEY')
MODEL = "typhoon-v1.5x-70b-instruct"
client = OpenAI(
   api_key=TYPHOON_API_KEY,
   base_url='https://api.opentyphoon.ai/v1'
)


class ChatTurn(TypedDict):
    role: str
    content: str


def get_open_ai_completion(
    prompt: str,
    model: str = MODEL,
    stream: bool = False,
    initial_message: list[ChatTurn]| None = None,
    temperature: float = 0
) -> str:
    if initial_message is None:
        initial_message = []
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=temperature, stream=stream
    )
    if not stream:
        return response.choices[0].message.content
    else:
        result = ""
        for chunk in response:
            content = chunk.choices[0].delta.content
            print(content, end="")
            if isinstance(content, str):
                result += content
        return result


def get_gemini_completion(
    prompt: str,
    stream: bool = False,
) -> str:
    # model = genai.GenerativeModel("gemini-1.5-pro")
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

In [17]:
print(f"Business type: {business_type}")

Business type: ประกอบด้วยกิจการที่ ปตท. ดำเนินการเอง ได้แก่ ธุรกิจก๊าซธรรมชาติ ธุรกิจระบบท่อส่งก๊าซธรรมชาติ ธุรกิจการค้าระหว่างประเทศธุรกิจใหม่และโครงสร้างพื้นฐาน และธุรกิจที่ลงทุนผ่านบริษัทย่อยและ\u002Fหรือกิจการที่ควบคุมร่วมกันและบริษัทร่วม (กลุ่มบริษัท) ได้แก่ ธุรกิจสำรวจและผลิตปิโตรเลียม ธุรกิจก๊าซธรรมชาติเหลว ธุรกิจปิโตรเคมีและการกลั่น ธุรกิจน้ำมันและค้าปลีก ธุรกิจไฟฟ้าและสาธารณูปการ ธุรกิจให้บริการ


In [18]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="Analyse this stock balance sheet in Thai",
)
Markdown(get_gemini_completion(prompt))

## วิเคราะห์งบดุลของบริษัท PTT (ปี 2562-2566)

งบดุลของบริษัท PTT แสดงให้เห็นถึงโครงสร้างทางการเงินของบริษัทในช่วงปี 2562-2566 โดยสรุปได้ดังนี้

**สินทรัพย์:**

* **สินทรัพย์รวม:** มีแนวโน้มเพิ่มขึ้นอย่างต่อเนื่องในทุกปี โดยในปี 2566 มีมูลค่า 1,117,390,962.4 ล้านบาท 
* **สินทรัพย์หมุนเวียน:** แสดงถึงความสามารถของบริษัทในการเปลี่ยนสินทรัพย์เป็นเงินสดได้อย่างรวดเร็ว มีมูลค่าเพิ่มขึ้นในปี 2565 และ 2566 แสดงถึงการดำเนินงานที่คล่องตัวมากขึ้น
    * **เงินสดและรายการเทียบเท่าเงินสด:** มีมูลค่าสูงขึ้นอย่างมีนัยสำคัญในปี 2566 โดยเฉพาะอย่างยิ่งในปี 2566
    * **ลูกหนี้การค้าและลูกหนี้หมุนเวียนอื่น:** มีมูลค่าสูงขึ้นในปี 2565 และ 2566 แสดงถึงยอดขายที่เพิ่มขึ้น
    * **สินค้าคงเหลือ:** มีมูลค่าเพิ่มขึ้นในปี 2566 บ่งบอกถึงการเติบโตของธุรกิจ
* **สินทรัพย์ไม่หมุนเวียน:** ส่วนใหญ่เป็นสินทรัพย์ถาวร เช่น ที่ดิน อาคาร และอุปกรณ์  แสดงให้เห็นการลงทุนในโครงสร้างพื้นฐานของบริษัท 
    * **เงินลงทุนในบริษัทย่อย บริษัทร่วม และการร่วมค้า:**  เป็นสัดส่วนที่สูงของสินทรัพย์ไม่หมุนเวียน แสดงถึงการลงทุนที่หลากหลายในธุรกิจต่างๆ
    * **ที่ดิน อาคาร และอุปกรณ์:**  มีมูลค่าลดลงเล็กน้อยในปี 2566 แสดงถึงการใช้ทรัพย์สินเพื่อการดำเนินงาน

**หนี้สิน:**

* **หนี้สินรวม:** มีแนวโน้มเพิ่มขึ้นในทุกปี โดยในปี 2566 มีมูลค่า 454,600,979.2 ล้านบาท 
* **หนี้สินหมุนเวียน:** ส่วนใหญ่เป็นหนี้สินที่เกี่ยวข้องกับการดำเนินงานประจำวันของบริษัท
    * **เจ้าหนี้การค้าและเจ้าหนี้หมุนเวียนอื่น:** มูลค่าลดลงในปี 2565 แต่เพิ่มขึ้นในปี 2566
    * **เงินกู้ยืมระยะสั้น:**  มีมูลค่าเพิ่มขึ้นอย่างมีนัยสำคัญในปี 2566
    * **ส่วนของหนี้สินระยะยาวส่วนที่ถึงกำหนดชำระภายในหนึ่งปี:** มีมูลค่าสูงขึ้นในปี 2566
* **หนี้สินไม่หมุนเวียน:** ส่วนใหญ่เป็นหนี้สินระยะยาว เช่น หนี้สินระยะยาว - ส่วนที่ถึงกำหนดชำระเกินกว่าหนึ่งปี 
    * **หนี้สินระยะยาว - ส่วนที่ถึงกำหนดชำระเกินกว่าหนึ่งปี:** มีมูลค่าเพิ่มขึ้นในปี 2565 และ 2566

**ส่วนของผู้ถือหุ้น:**

* **ส่วนของผู้ถือหุ้นรวม:**  มีมูลค่าเพิ่มขึ้นในทุกปี โดยในปี 2566 มีมูลค่า 662,789,983.3 ล้านบาท
    * **กำไร (ขาดทุน) สะสม:**  แสดงให้เห็นถึงผลประกอบการของบริษัท ซึ่งมีมูลค่าเพิ่มขึ้นในทุกปี บ่งบอกถึงการทำกำไรอย่างต่อเนื่อง
    * **องค์ประกอบอื่นของส่วนของผู้ถือหุ้น:**  แสดงถึงส่วนเกินจากมูลค่าหุ้นและการเปลี่ยนแปลงในบริษัทย่อย

**การวิเคราะห์เพิ่มเติม:**

* **อัตราส่วนทางการเงิน:** ควรวิเคราะห์อัตราส่วนทางการเงินที่เกี่ยวข้องกับการทำกำไร การชำระหนี้ และสภาพคล่อง เพื่อประเมินประสิทธิภาพการดำเนินงานของบริษัท
* **ปัจจัยภายนอก:** ควรพิจารณาปัจจัยภายนอก เช่น สถานการณ์เศรษฐกิจโลก ราคาพลังงาน ภาวะการเมือง 
* **การเปรียบเทียบกับคู่แข่ง:**  ควรเปรียบเทียบงบดุลของบริษัท PTT กับบริษัทอื่นๆ ในอุตสาหกรรมเดียวกัน เพื่อประเมินความแข็งแกร่งทางการเงิน

## สรุป:

งบดุลของบริษัท PTT แสดงให้เห็นถึงสถานะทางการเงินที่แข็งแกร่ง โดยมีสินทรัพย์รวมและส่วนของผู้ถือหุ้นเพิ่มขึ้นอย่างต่อเนื่อง ในขณะที่หนี้สินก็มีแนวโน้มเพิ่มขึ้นเช่นกัน อย่างไรก็ตาม ควรวิเคราะห์ข้อมูลเพิ่มเติม เช่น อัตราส่วนทางการเงิน ปัจจัยภายนอก และการเปรียบเทียบกับคู่แข่ง เพื่อให้ได้ภาพรวมที่สมบูรณ์ของสถานการณ์ทางการเงินของบริษัท

In [19]:
import matplotlib
import matplotlib.font_manager as fm
from os import path
if not path.exists('Sarabun-Regular.ttf'):
    !wget https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf -O Sarabun-Regular.ttf

if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

In [26]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought how this data can support the investors
2. Use Matplotlib to create some visualisations to ease investors. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix.""",
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)

SyntaxError: unterminated triple-quoted string literal (detected at line 327) (<string>, line 316)

In [27]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought about income structure of this company.
2. Use Plotly to create some visualisations to ease investors. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix.""",
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)


## บทวิเคราะห์ข้อมูลงบดุลของ บมจ. ปตท. (PTT)

จากข้อมูลงบดุลของ บมจ. ปตท. (PTT) พบว่า 

* **สินทรัพย์รวม**: มีแนวโน้มเพิ่มขึ้นอย่างต่อเนื่องในช่วง 5 ปีที่ผ่านมา แสดงให้เห็นว่า PTT ขยายธุรกิจและลงทุนอย่างต่อเนื่อง 
* **สินทรัพย์ไม่หมุนเวียน**: มีสัดส่วนสูงกว่าสินทรัพย์หมุนเวียนอย่างมาก แสดงให้เห็นว่า PTT มีการลงทุนในสินทรัพย์ถาวร เช่น ที่ดิน อาคาร และอุปกรณ์ เป็นจำนวนมาก 
* **หนี้สิน**: มีแนวโน้มเพิ่มขึ้น แต่ยังคงอยู่ในระดับที่ควบคุมได้ 
* **ส่วนของผู้ถือหุ้น**: มีแนวโน้มเพิ่มขึ้นอย่างต่อเนื่อง  แสดงให้เห็นว่า PTT มีการทำกำไรและผลตอบแทนที่ดี

**ข้อสรุป**: PTT มีโครงสร้างทางการเงินที่แข็งแกร่ง มีการลงทุนและขยายธุรกิจอย่างต่อเนื่อง และมีผลประกอบการที่แข็งแกร่ง  อย่างไรก็ตาม เพื่อการวิเคราะห์ที่ละเอียดลึกยิ่งขึ้น ควรพิจารณาข้อมูลงบกำไรขาดทุน และงบกระแสเงินสด รวมถึงปัจจัยอื่นๆ เช่น สภาพเศรษฐกิจและแนวโน้มอุตสาหกรรมพลังงาน


In [28]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Thought about income structure 2022 vs 2023 and write your in comment section
2. Use Plotly to create some visualisations to ease investors focusing on 2022 vs 2023. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix.""",
    response="Python code, You must give only the answer do not add prologue, prefix and suffix."
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)


## สรุปผลการวิเคราะห์งบดุล PTT ปี 2022 และ 2023

จากการวิเคราะห์งบดุลของ PTT พบว่า

- **ในปี 2023 ปริมาณเงินสดและรายการเทียบเท่าเงินสดของ PTT เพิ่มขึ้นอย่างมาก แต่จากข้อมูลนี้ไม่สามารถระบุได้ชัดเจนว่าเป็นผลมาจากการสร้างรายได้หรือปัจจัยอื่น ๆ**
- **ในปี 2023 สินทรัพย์หมุนเวียนเพิ่มขึ้น ซึ่งอาจเกิดจากปริมาณสินค้าคงเหลือที่เพิ่มขึ้น**
- **เงินลงทุนระยะยาวของบริษัทลดลงในปี 2023 ซึ่งอาจบ่งชี้ถึงการเปลี่ยนแปลงกลยุทธ์การลงทุน**
- **หนี้สินระยะสั้นของ PTT เพิ่มขึ้นในปี 2023 ขณะที่หนี้สินระยะยาวลดลง แสดงให้เห็นถึงการเปลี่ยนแปลงกลยุทธ์ทางการเงินหรือเน้นการกู้ยืมระยะสั้น**

ข้อมูลดังกล่าวควรนำไปพิจารณาควบคู่กับข้อมูลอื่น ๆ เช่น งบกำไรขาดทุนและงบกระแสเงินสด เพื่อให้ได้ภาพรวมที่ครบถ้วนของสถานะการเงินของ PTT 


# CO-STAR prompting

Ref: https://medium.com/@frugalzentennial/unlocking-the-power-of-costar-prompt-engineering-a-guide-and-example-on-converting-goals-into-dc5751ce9875


```
# Context

I am analyzing a company's financial performance. The data includes the balance sheet, income statement, and cash flow statement for each year. I am interested in understanding key financial metrics and trends, such as profitability, liquidity, and solvency, as well as identifying any significant changes in financial health.

Stock name: ${SYMBOL}
Business type: ${business_type}
Balance sheet:
${sheet}

----

# Objective

The goal is to generate insights on the company's financial performance, including:
{task}

----

# Style

Write in an informative and instructional style

---

# Tone
The tone should be financial reporting with your suggestion.

---

# Audience

Individuals interested in company's finanical performance

---

# Response

${response}
```

In [30]:
co_star_template = Template("""
# Context

I am analyzing a company's financial performance.
The data includes the balance sheet, income statement, and cash flow statement for each year.
I am interested in understanding key financial metrics and trends, such as profitability, liquidity, and solvency, as well as identifying any significant changes in financial health.

Stock name: ${SYMBOL}
Business type: ${business_type}
Balance sheet:
${sheet}

----

# Objective

The goal is to generate insights on the company's financial performance, including:
{task}

----

# Style

Write in an informative and instructional style

---

# Tone
The tone should be financial reporting with your suggestion.

---

# Audience

Individuals interested in company's finanical performance

---

# Response

${response}
""")

In [31]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""
# Objective:
Answer the following questions:

2 วิเคราะห์หนี้สิน
2.1 ดูโครงสร้างหข้องหนี้สินว่าสอดคล้องกับโครงสร้างสินทรัพย์หรือไม่
2.1.1ถ้าบริษัทที่มีสินทรัพย์หมุนเวียนมากกว่าสินทรัพย์ไม่หมุนเวียน หนี้สินหมุนเวียนจะมากกว่าหนี้สินไม่หมุนเวียน แสดงว่าโครงสร้างหนี้สินสอดคล้องกับสินทรัพย์
2.1.2ถ้าบริษัทที่มีสินทรัพย์หมุนเวียนน้อยกว่าสินทรัพย์ไม่หมุนเวียน หนี้สินหมุนเวียนจะน้อยกว่าหนี้สินไม่หมุนเวียน แสดงว่าโครงสร้างหนี้สินสอดคล้องกับสินทรัพย์
2.1.3 ถ้าบริษัทมีสินทรัพย์ไม่หมุนเวียนเยอะ แต่หนี้ส่วนใหญ่เป็นหนี้ระยะสั้น ให้ระวังปัญหาหนี้สิน เนื่องจากใช้เงินผิดประเภทกู้ระยะสั้นมาลงทุนระยะยาว
2.2 ให้ระวังบริษัทที่มีส่วนของหนี้สินเพิ่มขึ้นเร็วกว่าส่วนของผู้ถือหุ้นที่เพิ่มขึ้น อาจมีปัญหาหนี้สินได้ในอนาคต
2.3 วิเคราะห์ส่วนของผู้ถือหุ้น
2.3.1 ให้ระวังบริษัทที่ขาดทุนสะสมจำนวนมากเมื่อเทียบกับทุนจดทะเบียน แสดงว่าในอดีตที่ผ่านมา มีผลประกอบการขาดทุนเป็นจำนวนมาก
2.3.2 ให้ระวังบริษัทที่มี ตราสารหนี้เสมือนทุน จำนวนมากเมื่อเทียบกับส่วนของผู้ถือหุ้น เพราะจริงๆแล้วเป็นหนี้สินไม่ใช่ส่วนของผู้ถือหุ้น
""",
    response="Markdown in Thai language"
)

output = get_gemini_completion(prompt)
Markdown(output)

## วิเคราะห์หนี้สินของ PTT

จากข้อมูลงบดุลของ PTT  ปี 2019-2023 เราจะวิเคราะห์หนี้สินตามหัวข้อต่อไปนี้:

**2.1 โครงสร้างหนี้สินสอดคล้องกับโครงสร้างสินทรัพย์หรือไม่**

| ปี | สินทรัพย์หมุนเวียน | สินทรัพย์ไม่หมุนเวียน | หนี้สินหมุนเวียน | หนี้สินไม่หมุนเวียน |
|:---|:---|:---|:---|:---|
| 2019 | 221,778,155.2 | 667,898,035.8 | 160,721,684.5 | 121,647,772.8 |
| 2020 | 204,888,987.6 | 694,499,274.9 | 107,260,505.1 | 169,045,268.3 |
| 2021 | 210,083,620.7 | 794,822,290.8 | 156,317,823.3 | 211,081,342.2 |
| 2022 | 291,605,069.7 | 790,166,731.2 | 205,765,734.1 | 247,212,521.7 |
| 2023 | 345,130,403.3 | 772,260,559.1 | 238,117,927.4 | 216,483,051.7 |

จากตารางข้างต้น เราสังเกตเห็นว่า:

* PTT มีสินทรัพย์ไม่หมุนเวียนมากกว่าสินทรัพย์หมุนเวียนในทุกปี  
* หนี้สินไม่หมุนเวียนมากกว่าหนี้สินหมุนเวียนในทุกปี 
*  **แสดงว่าโครงสร้างหนี้สินสอดคล้องกับสินทรัพย์** 

**2.2 ส่วนของหนี้สินเพิ่มขึ้นเร็วกว่าส่วนของผู้ถือหุ้นที่เพิ่มขึ้นหรือไม่**

| ปี | หนี้สินรวม | ส่วนของผู้ถือหุ้น | อัตราส่วนหนี้สินต่อทุน |
|:---|:---|:---|:---|
| 2019 | 282,369,457.3 | 607,306,733.7 | 0.46 |
| 2020 | 276,305,773.4 | 623,082,489.1 | 0.44 |
| 2021 | 367,399,165.6 | 637,506,745.9 | 0.58 |
| 2022 | 452,978,255.8 | 628,793,545.2 | 0.72 |
| 2023 | 454,600,979.2 | 662,789,983.3 | 0.69 |

จากตารางข้างต้น เราสังเกตเห็นว่า:

* หนี้สินรวมเพิ่มขึ้นในทุกปี ยกเว้นปี 2020  
* ส่วนของผู้ถือหุ้นเพิ่มขึ้นในทุกปี
* อัตราส่วนหนี้สินต่อทุนเพิ่มขึ้นในปี 2021, 2022, 2023  และลดลงในปี 2020  

แม้ว่าส่วนของหนี้สินเพิ่มขึ้นในบางปี แต่ส่วนของผู้ถือหุ้นก็เพิ่มขึ้นเช่นกัน และอัตราส่วนหนี้สินต่อทุนไม่เพิ่มขึ้นอย่างรวดเร็ว แสดงว่า **PTT ไม่มีความเสี่ยงสูงจากปัญหาหนี้สินในอนาคต**

**2.3 วิเคราะห์ส่วนของผู้ถือหุ้น**

| ปี | กำไรสะสม | ทุนจดทะเบียน | อัตราส่วนกำไรสะสมต่อทุนจดทะเบียน | ตราสารหนี้เสมือนทุน |
|:---|:---|:---|:---|:---|
| 2019 | 548,106,139.4 | 28,562,996.2 | 19.18 | - |
| 2020 | 563,750,104.9 | 28,562,996.2 | 19.74 | - |
| 2021 | 574,485,162.1 | 28,562,996.2 | 20.11 | - |
| 2022 | 567,779,131.7 | 28,562,996.2 | 19.89 | - |
| 2023 | 602,959,929.8 | 28,562,996.2 | 21.14 | - |

จากตารางข้างต้น เราสังเกตเห็นว่า:

* กำไรสะสมของ PTT มีจำนวนมากเมื่อเทียบกับทุนจดทะเบียน 
* อัตราส่วนกำไรสะสมต่อทุนจดทะเบียนอยู่ในระดับสูง  
* PTT ไม่มีตราสารหนี้เสมือนทุน  

แสดงว่า **PTT มีความมั่นคงทางการเงิน และมีประวัติผลประกอบการที่ดี** 

**สรุป**

จากการวิเคราะห์หนี้สินของ PTT พบว่า:

* โครงสร้างหนี้สินสอดคล้องกับสินทรัพย์ 
* PTT ไม่มีความเสี่ยงสูงจากปัญหาหนี้สินในอนาคต
* PTT มีความมั่นคงทางการเงิน และมีประวัติผลประกอบการที่ดี

**หมายเหตุ:** 

* การวิเคราะห์นี้เป็นเพียงการวิเคราะห์เบื้องต้น 
* ควรศึกษาข้อมูลเพิ่มเติมเกี่ยวกับ PTT เช่น งบกำไรขาดทุน งบกระแสเงินสด และรายงานของบริษัท  
* ควรปรึกษาผู้เชี่ยวชาญด้านการเงินก่อนตัดสินใจลงทุนใน PTT 
